In [1]:
# app.py
import streamlit as st
import pandas as pd
import joblib
import plotly.express as px
import time

# File paths
DATA_PATH = r"C:\Users\sagni\Downloads\Spacepath Debris Detection\aircraft-data_nov_dec.csv"
MODEL_PATH = r"C:\Users\sagni\Downloads\Spacepath Debris Detection\isolation_forest_model.pkl"
SCALER_PATH = r"C:\Users\sagni\Downloads\Spacepath Debris Detection\scaler.pkl"

# Load model and scaler
@st.cache_resource
def load_model():
    model = joblib.load(MODEL_PATH)
    scaler = joblib.load(SCALER_PATH)
    return model, scaler

# Load data and preprocess
@st.cache_data
def load_data():
    df = pd.read_csv(DATA_PATH)
    df = df.dropna(subset=['alt', 'mph', 'lat', 'long'])
    return df

# Inference
def predict_anomalies(df, model, scaler):
    X = scaler.transform(df[['alt', 'mph', 'lat', 'long']])
    df['anomaly_score'] = model.decision_function(X)
    df['is_anomaly'] = model.predict(X)
    df['is_anomaly'] = df['is_anomaly'].map({1: 0, -1: 1})
    return df

# UI
st.set_page_config(layout="wide", page_title="✈ Real-Time Aircraft Anomaly Detection")

st.title("🛫 Real-Time Aircraft Anomaly Monitor (Isolation Forest)")
model, scaler = load_model()
df = load_data()

# Simulate streaming with slider
chunk_size = 100
max_step = len(df) // chunk_size
step = st.slider("Simulation Step", 1, max_step, 1)
live_df = df.iloc[:step * chunk_size].copy()

# Predict anomalies on chunk
live_df = predict_anomalies(live_df, model, scaler)

# Map
fig = px.scatter_mapbox(
    live_df,
    lat="lat",
    lon="long",
    color=live_df['is_anomaly'].map({0: "Normal", 1: "Anomaly"}),
    hover_data=["flight", "alt", "mph"],
    zoom=3,
    height=600
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Layout
col1, col2 = st.columns([2, 1])
with col1:
    st.plotly_chart(fig, use_container_width=True)

with col2:
    st.subheader("✈ Anomalous Flights")
    st.dataframe(live_df[live_df['is_anomaly'] == 1][['flight', 'alt', 'mph', 'lat', 'long']])

st.caption("🔁 Use the slider to simulate real-time monitoring")


2025-08-03 17:19:59.422 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-08-03 17:19:59.424 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 17:19:59.426 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 17:20:00.074 
  command:

    streamlit run C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-03 17:20:00.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 17:20:00.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 17:20:00.077 Thread 'MainThread': missing ScriptRunContext! This warn

DeltaGenerator()